# 🎯 Let's Talk Model Selection & Synthetic Data

## 🛒 The Zava Scenario

Cora needs to recommend home improvement products from Zava's catalog effectively. But which AI model should power Cora? Should we use GPT-4.1 for its advanced reasoning, or GPT-4o-mini for cost efficiency?

**The Challenge**: Before deploying Cora to production, we need to evaluate different models to find the best balance of quality, speed, and cost. We also need realistic test data to compare how each model performs with customer queries.

## What You'll Learn

In this section, you'll understand:

1. **How to choose between different AI models** for your use case
2. **What synthetic datasets are** and why they're valuable for testing
3. **How to generate query-response pairs** for evaluation
4. **What RAG (Retrieval-Augmented Generation) is** and how it improves responses
5. **Key evaluation metrics** to compare model performance

## Why This Matters

Selecting the right model is critical because it impacts:
- **Response quality** - How accurate and helpful Cora's recommendations are
- **Cost** - Different models have different pricing structures
- **Latency** - How quickly Cora responds to customers
- **Scalability** - Whether the solution can handle production traffic

Let's explore how to make informed model decisions using synthetic data and evaluation.

---

## Why Model Selection Matters

Not all language models are created equal. Different models have different strengths:

| Model | Best For | Trade-offs |
|-------|----------|------------|
| **GPT-4o** | Complex reasoning, multimodal tasks | Higher cost, slower |
| **GPT-4o-mini** | High-volume, speed-critical applications | Less nuanced, faster |
| **GPT-4.1** | General-purpose, strong performance | Balanced cost/quality |
| **Fine-tuned models** | Domain-specific tasks, consistent style | Requires training data |

**Key factors in model selection:**

### 1. Task Complexity
- **Simple Q&A**: GPT-4o-mini works great
- **Multi-step reasoning**: GPT-4o provides better results
- **Domain expertise**: Consider fine-tuning

### 2. Cost Constraints
Models charge per token (input + output):
- GPT-4o-mini: Lower per-token cost, higher volume capability
- GPT-4o: Higher per-token cost, better quality

**Example calculation:**
```
10,000 customer queries/day
Average 500 tokens per conversation

Model A (smaller): Lower cost per token
Model B (larger):  Higher cost per token

Choosing the right model can lead to significant cost savings at scale.

Note: See Azure OpenAI Pricing for current rates:
https://azure.microsoft.com/en-us/pricing/details/cognitive-services/openai-service/
```

### 3. Latency Requirements
- **Real-time chat**: GPT-4o-mini (faster responses)
- **Background processing**: GPT-4o (quality over speed)

### 4. Accuracy Requirements
- **Factual precision critical** (e.g., medical, legal): GPT-4o + RAG
- **General assistance** (e.g., recommendations): GPT-4o-mini sufficient


**The Challenge:** How do you know which model is best for *your* use case?**The Solution:** Systematic evaluation with test datasets.


---

## What are Synthetic Datasets?

**Synthetic datasets** are artificially generated test data that simulate real-world scenarios.

### Why Generate Synthetic Data?

**Problem:** You need test data before you have real customer conversations.

**Solutions:**
1. ❌ Wait for real data → Can't test until production
2. ❌ Manually write test cases → Time-consuming, limited coverage
3. ✅ Generate synthetic data → Fast, scalable, covers edge cases

### Benefits of Synthetic Data

**1. Early Testing**
- Test models before deployment
- Catch issues in development
- Iterate quickly on improvements

**2. Comprehensive Coverage**
- Generate hundreds of test cases quickly
- Cover edge cases humans might miss
- Test different customer intents and phrasings

**3. Privacy & Safety**
- No real customer data needed
- Safe for development/testing
- Compliance-friendly

**4. Cost-Effective**
- Faster than manual test creation
- Cheaper than waiting for real data
- Reusable across iterations

### Example: Zava Product Questions

**Real customer questions** (would take months to collect):
```
"What paint do you have for exterior wood?"
"I need a drill for concrete, what do you recommend?"
"Do you have any eco-friendly paint options?"
"What's the difference between latex and oil-based paint?"
```

**Synthetic generation** (created in minutes):
```python
# Azure AI Simulator generates similar questions from product catalog
simulator.generate_queries(
    source="product_catalog",
    count=100,
    variety=["product_search", "recommendations", "comparisons"]
)
```

**Result:** 100 realistic customer questions in seconds, covering diverse intents.

---

## Azure AI Evaluation Simulator

The **Azure AI Evaluation Simulator** is a tool for generating synthetic query-response pairs based on your data sources.

### How It Works

```
Your Data Sources           Simulator              Synthetic Dataset
                                ↓
┌─────────────────┐      ┌──────────────┐      ┌──────────────────┐
│ Product Catalog │  →   │  AI Simulator │  →   │ Query-Response   │
│ Documentation   │      │               │      │ Pairs (JSONL)    │
│ FAQs            │      │  Uses LLM to  │      │                  │
│ Knowledge Base  │      │  generate     │      │ 100+ realistic   │
└─────────────────┘      │  realistic    │      │ test examples    │
                         │  questions     │      └──────────────────┘
                         └──────────────┘
```

### What Gets Generated

**Query-Response Pairs** in JSONL format:

```json
{
  "query": "What paint is best for exterior wood?",
  "response": "For exterior wood, we recommend our Premium Exterior Paint (SKU: PFIP000002)...",
  "context": "Product: Premium Exterior Paint, Category: Paint & Finishes",
  "intent": "product_recommendation"
}
```

Each pair includes:
- **Query**: A customer question
- **Response**: Expected answer (generated from your data)
- **Context**: Source information used
- **Intent**: Type of question (optional metadata)

### Key Components

**1. Data Source Connection**
Connect to your knowledge base:
```python
# Example: Azure AI Search
search_client = SearchClient(
    endpoint=AZURE_SEARCH_ENDPOINT,
    index_name="products",
    credential=AzureKeyCredential(AZURE_SEARCH_API_KEY)
)
```

**2. RAG Application Callback**
Define how to retrieve information:
```python
def query_product_info(query: str) -> str:
    # Search product catalog
    results = search_client.search(query, top=3)
    # Format results
    return formatted_results
```

**3. Simulation Configuration**
Generate queries:
```python
simulator = Simulator(model_config=model_config)

outputs = simulator.generate(
    target=query_product_info,  # Your retrieval function
    num_queries=100,            # How many to generate
    max_conversation_turns=1    # Single Q&A or multi-turn
)
```

### Output Format: JSONL

**JSONL** (JSON Lines) = One JSON object per line

```jsonl
{"query": "What drill bits do you have?", "response": "We offer..."}
{"query": "Best paint for kitchens?", "response": "For kitchens..."}
{"query": "Do you have eco-friendly products?", "response": "Yes, we have..."}
```

**Why JSONL?**
- Easy to stream and process line-by-line
- Standard format for ML/AI tools
- Efficient for large datasets
- Compatible with evaluation libraries

---

## RAG: Retrieval-Augmented Generation

**RAG** is a technique that improves AI responses by retrieving relevant information before generating answers.

### The Problem Without RAG

**Scenario:** Customer asks "What is SKU PFIP000002?"

**Without RAG (model alone):**
```
Model: "I don't have information about specific SKUs in my training data."
```

The model doesn't know your specific products.

### The Solution: RAG

**With RAG:**
```
1. Retrieve: Search product database for "PFIP000002"
   → Found: "Premium Exterior Paint, $45.99, In Stock"
   
2. Augment: Add retrieved info to prompt
   "Based on this product info: [Premium Exterior Paint...], answer the question"
   
3. Generate: Model creates response
   → "SKU PFIP000002 is Premium Exterior Paint, priced at $45.99 and currently in stock."
```

**Result:** Accurate, factual response grounded in real data.

### How RAG Works

```
Customer Question
      ↓
┌─────────────────┐
│  1. Retrieve    │  Search knowledge base
│                 │  (Azure AI Search, Vector DB, etc.)
└────────┬────────┘
         ↓
   Retrieved Context
         ↓
┌─────────────────┐
│  2. Augment     │  Combine question + context in prompt
│                 │  "Based on: [context], answer: [question]"
└────────┬────────┘
         ↓
   Enhanced Prompt
         ↓
┌─────────────────┐
│  3. Generate    │  LLM creates response using context
│                 │  
└────────┬────────┘
         ↓
   Factual Response
```

### RAG vs. Fine-Tuning vs. Prompting

| Approach | Use When | Pros | Cons |
|----------|----------|------|------|
| **RAG** | Data changes frequently | Always up-to-date, factual | Requires search infrastructure |
| **Fine-Tuning** | Style/tone consistency needed | Efficient, no retrieval needed | Static knowledge, requires retraining |
| **Prompting** | Simple tasks, static info | Fast, no infrastructure | Limited by context window |

**For Cora:** Use RAG for product info (changes frequently) + fine-tuning for tone (static style).

### Benefits for Model Evaluation

When generating synthetic datasets with the simulator:
- **RAG ensures grounded responses** - Answers based on real product data
- **Realistic test cases** - Questions reflect actual product catalog
- **Measurable accuracy** - Can verify responses against source data

This is why the simulator uses a RAG callback - it generates test data that's realistic and verifiable.

---

## Key Concepts: Queries, Responses, and Pairs

### Query

A **query** is a question or request from the user.

**Examples:**
- "What paint do you have for exterior wood?"
- "I need a drill, what do you recommend?"
- "Is SKU PFIP000002 in stock?"

**Query characteristics:**
- **Intent**: What the user wants (search, recommendation, fact check)
- **Complexity**: Simple vs. multi-part questions
- **Specificity**: Broad ("what paint?") vs. specific ("is PFIP000002 available?")

### Response

A **response** is the answer generated by the AI model.

**Examples:**
```
Query: "What paint do you have for exterior wood?"

Response: "For exterior wood projects, I recommend our Premium Exterior Paint 
(SKU: PFIP000002). It's weather-resistant, durable, and available in multiple 
colors. Currently priced at $45.99 with 75 units in stock."
```

**Quality factors:**
- **Accuracy**: Factually correct
- **Completeness**: Answers the full question
- **Relevance**: Stays on topic
- **Helpfulness**: Provides useful information
- **Tone**: Matches brand voice (polite, professional)

### Query-Response Pair

A **pair** combines a query with its expected response for testing.

**Format:**
```json
{
  "query": "What paint is best for exterior wood?",
  "response": "For exterior wood, I recommend Premium Exterior Paint (SKU: PFIP000002)...",
  "ground_truth": "Premium Exterior Paint",
  "context": "Products: PFIP000002, PFIP000003, PFIP000005"
}
```

**Why pairs?**
- **Baseline for comparison**: Expected vs. actual responses
- **Reproducible testing**: Same queries across model versions
- **Quality metrics**: Measure how well responses match expectations

### Evaluation Workflow

```
1. Generate Pairs (Simulator)
   → 100 query-response pairs from product catalog

2. Test Model A (GPT-4o-mini)
   → Run 100 queries through model
   → Collect 100 responses

3. Test Model B (GPT-4o)
   → Run same 100 queries through different model
   → Collect 100 responses

4. Compare Results
   → Which model's responses better match expected responses?
   → Which is more accurate, helpful, relevant?

5. Select Winner
   → Choose model based on metrics
```

---

## Evaluation Metrics for Model Selection

How do you measure which model is better? Use these metrics:

### 1. Groundedness

**What it measures:** Are responses based on retrieved context (not hallucinated)?

**Example:**
```
Context: "Premium Exterior Paint costs $45.99"

Good (grounded): "Premium Exterior Paint is priced at $45.99"
Bad (not grounded): "Premium Exterior Paint costs around $40"
```

**Score:** 0 to 5 (5 = fully grounded in context)

### 2. Relevance

**What it measures:** Does the response address the query?

**Example:**
```
Query: "What paint is best for exterior wood?"

Good (relevant): "For exterior wood, Premium Exterior Paint is ideal..."
Bad (irrelevant): "We have many paint options in different colors..."
```

**Score:** 0 to 5 (5 = perfectly relevant)

### 3. Coherence

**What it measures:** Is the response well-structured and logical?

**Example:**
```
Good (coherent): "Premium Exterior Paint is durable and weather-resistant, 
making it perfect for outdoor wood surfaces."

Bad (incoherent): "Paint wood exterior durable Premium weather yes outdoor."
```

**Score:** 0 to 5 (5 = perfectly coherent)

### 4. Fluency

**What it measures:** Is the language natural and grammatically correct?

**Example:**
```
Good (fluent): "I recommend Premium Exterior Paint for your project."
Bad (not fluent): "I recommend you Premium Exterior Paint is for project."
```

**Score:** 0 to 5 (5 = perfect grammar and naturalness)

### 5. Similarity (to Expected Response)

**What it measures:** How close is the actual response to the expected response?

**Measured by:**
- Cosine similarity (embedding vectors)
- BLEU score (text overlap)
- Semantic similarity (meaning)

**Example:**
```
Expected: "Premium Exterior Paint costs $45.99 and is in stock."
Actual:   "Our Premium Exterior Paint is priced at $45.99 with availability."

Similarity: 0.92 (very similar)
```

### Combining Metrics

**Model evaluation scorecard:**

| Metric | GPT-4o-mini | GPT-4o | Winner |
|--------|-------------|--------|--------|
| Groundedness | 4.2 | 4.8 | GPT-4o |
| Relevance | 4.5 | 4.7 | GPT-4o |
| Coherence | 4.3 | 4.6 | GPT-4o |
| Fluency | 4.6 | 4.7 | GPT-4o |
| Similarity | 0.85 | 0.91 | GPT-4o |
| **Avg Latency** | **800ms** | **1200ms** | **GPT-4o-mini** |
| **Cost/1K queries** | **Lower** | **Higher** | **GPT-4o-mini** |

**Note:** For current pricing, see [Azure OpenAI Pricing](https://azure.microsoft.com/pricing/details/cognitive-services/openai-service/)

**Decision:** 
- If quality is critical → Choose GPT-4o

- If speed/cost is critical → Choose GPT-4o-mini- Hybrid: Use GPT-4o-mini for simple queries, GPT-4o for complex ones

---

## The Model Selection Process

### Step 1: Define Requirements

**Ask yourself:**
- What is my task? (Q&A, recommendations, complex reasoning)
- What quality level is acceptable?
- What's my budget?
- What latency is acceptable?
- How many queries per day?

**Example for Cora:**
- Task: Product Q&A, inventory checks
- Quality: High accuracy required (factual info)
- Budget: Moderate (thousands of queries/day)
- Latency: < 2 seconds preferred
- Volume: ~5,000 queries/day

### Step 2: Generate Test Dataset

**Use Azure AI Simulator:**
```python
# Generate 100 query-response pairs from product catalog
dataset = simulator.generate(
    target=rag_callback,
    num_queries=100
)

# Save to JSONL
with open("test_dataset.jsonl", "w") as f:
    for item in dataset:
        f.write(json.dumps(item) + "\n")
```

**Result:** 100 realistic customer questions with expected answers

### Step 3: Test Candidate Models

**Run each model on the test dataset:**

```python
# Test GPT-4o-mini
results_mini = []
for item in test_dataset:
    response = model_mini.query(item["query"])
    results_mini.append({
        "query": item["query"],
        "response": response,
        "expected": item["response"]
    })

# Test GPT-4o
results_4o = []
for item in test_dataset:
    response = model_4o.query(item["query"])
    results_4o.append({
        "query": item["query"],
        "response": response,
        "expected": item["response"]
    })
```

### Step 4: Evaluate Results

**Use evaluation metrics:**

```python
from azure.ai.evaluation import evaluate

# Evaluate GPT-4o-mini
eval_mini = evaluate(
    data=results_mini,
    evaluators={
        "groundedness": groundedness_evaluator,
        "relevance": relevance_evaluator,
        "coherence": coherence_evaluator
    }
)

# Evaluate GPT-4o
eval_4o = evaluate(
    data=results_4o,
    evaluators={...}
)
```

### Step 5: Compare and Decide

**Create comparison:**

```python
comparison = {
    "GPT-4o-mini": {
        "quality": eval_mini.average_scores,
        "cost": 0.50,
        "latency": 800
    },
    "GPT-4o": {
        "quality": eval_4o.average_scores,
        "cost": 2.50,
        "latency": 1200
    }
}
```

**Decision framework:**

```
If quality_difference < 0.3 AND cost_difference > 2x:
    → Choose cheaper model (GPT-4o-mini)
    
If quality_difference > 0.5:
    → Choose better model (GPT-4o)
    
Else:
    → Consider hybrid approach
```

### Step 6: Iterate

- Test with more data points
- Try different prompts
- Consider fine-tuning
- Re-evaluate periodically

---

## Best Practices

### 1. Generate Diverse Test Cases

```python
# Good - covers different intents
queries = [
    "What paint is best for exterior wood?",      # Recommendation
    "Is SKU PFIP000002 in stock?",                # Fact check
    "Compare latex vs oil-based paint",           # Comparison
    "I need eco-friendly options",                # Filtered search
    "What's the price of Premium Exterior Paint?" # Specific fact
]

# Less effective - repetitive
queries = [
    "What paint do you have?",
    "Do you sell paint?",
    "Tell me about paint",
    # All similar intent
]
```

### 2. Use Realistic Phrasings

Generate queries that match how real customers talk:

```python
# Realistic
"I'm painting my deck, what should I use?"
"Need something for outdoor wood"
"Best paint for weather resistance?"

# Too formal (less realistic)
"Please provide recommendations for exterior wood coating solutions"
```

### 3. Include Edge Cases

```python
# Test edge cases
"Do you have paint?" # Vague
"I need PFIP000002 but in blue" # Specific constraint
"What's the cheapest paint?" # Price-focused
"" # Empty query
"ajshdkajhsd" # Gibberish
```

### 4. Balance Dataset Size

- **Too small** (< 20 queries): Not representative
- **Good** (50-100 queries): Balanced coverage
- **Large** (500+ queries): Comprehensive, but slower/costlier to run

**Start with 50-100, expand if needed**

### 5. Version Your Datasets

```python
# Save with version numbers
"test_dataset_v1.jsonl"  # Initial
"test_dataset_v2.jsonl"  # Added edge cases
"test_dataset_v3.jsonl"  # Added multi-turn conversations
```

This lets you compare model performance over time.

---

## Terminology Quick Reference

| Term | Simple Definition |
|------|-------------------|
| **Synthetic Dataset** | Artificially generated test data simulating real scenarios |
| **Query** | A question or request from the user |
| **Response** | The answer generated by the AI model |
| **Query-Response Pair** | A test case combining a query with expected response |
| **RAG** | Retrieval-Augmented Generation - retrieve context before generating |
| **JSONL** | JSON Lines format - one JSON object per line |
| **Groundedness** | Metric measuring if response is based on provided context |
| **Relevance** | Metric measuring if response addresses the query |
| **Coherence** | Metric measuring if response is well-structured |
| **Fluency** | Metric measuring if response has natural language quality |
| **Similarity** | Metric comparing actual vs. expected responses |
| **Simulator** | Tool that generates synthetic test data |
| **Latency** | Time taken to generate a response |
| **Token** | Unit of text (~4 characters) used for billing |

---

## What's Next?

Now that you understand model selection concepts, you're ready to generate and evaluate test datasets!

### Hands-On Notebooks in This Section

- **`21-simulate-dataset.ipynb`** - Generate synthetic test data with Azure AI Simulator
  - Connect to Azure AI Search
  - Create RAG application callback
  - Generate query-response pairs
  - Save datasets in JSONL format

- **`22-evaluate-models.ipynb`** - Compare different models using your test dataset
  - Run queries through multiple models
  - Calculate evaluation metrics
  - Compare quality, cost, and latency
  - Make data-driven model selection

### Recommended Learning Path

1. **Start here** → Understand concepts (this notebook)
2. **Next** → Generate test dataset (`21-simulate-dataset.ipynb`)
3. **Then** → Evaluate models (`22-evaluate-models.ipynb`)
4. **After** → Move to customization labs (fine-tuning, distillation)
5. **Finally** → Deploy and monitor your chosen model

---

## Further Reading

For deeper understanding:

- **[Azure AI Evaluation SDK](https://learn.microsoft.com/azure/ai-studio/how-to/develop/evaluate-sdk)** - Official evaluation guide
- **[Azure AI Simulator](https://learn.microsoft.com/azure/ai-studio/how-to/develop/simulator-interaction-data)** - Generate synthetic datasets
- **[RAG Overview](https://learn.microsoft.com/azure/ai-studio/concepts/retrieval-augmented-generation)** - Retrieval-Augmented Generation concepts
- **[Model Selection Guide](https://learn.microsoft.com/azure/ai-services/openai/concepts/models)** - Choosing the right model
- **[Evaluation Metrics](https://learn.microsoft.com/azure/ai-studio/concepts/evaluation-metrics-built-in)** - Understanding quality metrics

---

Ready to generate test data? Open `21-simulate-dataset.ipynb` to get started! 🚀